# Data Quality & Sanity Check
1. Total rows and Distinct Accident rows
2. Missing values (NaN + blanks)

### Total rows and Distinct Accident Rows

In [1]:
import pandas as pd

# Load the all the dataset from Dataset folder
def load_dataset(file_name):
    return pd.read_csv(f'Dataset/{file_name}', low_memory=False)

file_list = [
    'ACCIDENT.csv',
    'ACCIDENT_CHAINAGE.csv',
    'ACCIDENT_EVENT.csv',
    'ACCIDENT_LOCATION.csv',
    'ATMOSPHERIC_COND.csv',
    'NODE.csv',
    'NODE_ID_COMPLEX_INT_ID.csv',
    'PERSON.csv',
    'ROAD_SURFACE_COND.csv',
    'SUBDCA.csv',
    'VEHICLE.csv'
]
# Load all datasets into a dictionary
datasets = {file_name.split('.')[0]: load_dataset(file_name) for file_name in file_list}
# Save the datasets to a dictionary for further processing
datasets = {
    'ACCIDENT': load_dataset('ACCIDENT.csv'),
    'ACCIDENT_CHAINAGE': load_dataset('ACCIDENT_CHAINAGE.csv'),
    'ACCIDENT_EVENT': load_dataset('ACCIDENT_EVENT.csv'),
    'ACCIDENT_LOCATION': load_dataset('ACCIDENT_LOCATION.csv'),
    'ATMOSPHERIC_COND': load_dataset('ATMOSPHERIC_COND.csv'),
    'NODE': load_dataset('NODE.csv'),
    'NODE_ID_COMPLEX_INT_ID': load_dataset('NODE_ID_COMPLEX_INT_ID.csv'),
    'PERSON': load_dataset('PERSON.csv'),
    'ROAD_SURFACE_COND': load_dataset('ROAD_SURFACE_COND.csv'),
    'SUBDCA': load_dataset('SUBDCA.csv'),
    'VEHICLE': load_dataset('VEHICLE.csv')
}

In [2]:
summary_total = []
for name, df in datasets.items():
    if "ACCIDENT_NO" in df.columns:
        total_rows = len(df)
        distinct_accidents = df["ACCIDENT_NO"].nunique()
        one_to_one = total_rows == distinct_accidents
    else:
        total_rows = len(df)
        distinct_accidents = len(df.drop_duplicates())
        one_to_one = 'No ACCIDENT_NO'
    summary_total.append(
        {
            "Dataset": name,
            "Total Rows": total_rows,
            "Distinct Rows": distinct_accidents,
            "One to One with Accident": one_to_one
        }
    )
summary_total_df = pd.DataFrame(summary_total)
display(summary_total_df)

,Dataset,Total Rows,Distinct Rows,One to One with Accident
0,ACCIDENT,203708,203708,True
1,ACCIDENT_CHAINAGE,173557,173557,No ACCIDENT_NO
2,ACCIDENT_EVENT,326638,203708,False
3,ACCIDENT_LOCATION,203708,203708,True
4,ATMOSPHERIC_COND,206958,203708,False
5,NODE,221797,202854,False
6,NODE_ID_COMPLEX_INT_ID,203996,203708,False
7,PERSON,490948,203708,False
8,ROAD_SURFACE_COND,205030,203708,False
9,SUBDCA,290966,197897,False


As we can see in the table, total Accidents should be 203,708, the reason why some of table's total rows and distinct accident rows didn't match:
1. ACCIDENT_CHAINAGE contains location reference information for accidents, and it didn't have ACCIDENT_NO column
2. ACCIDENT_EVENT: Each accident may have multiple event (e.g., collision, vehicles involved) recorded separately.
3. ATMOSPHERIC_COND: Some accident may face multiple weather conditions recorded separately.
4. NODE: Not all crashes could be geo located successfully.
5. NODE_ID_COMPLEX_INT_ID: Expected less as it is only applicable to intersections classified as complex intersection. 
6. PERSON: Each accident can involve multiple people (e.g., driver, passenger)
7. ROAD_SURFACE_COND: Some accident may face multiple road surface conditions.
8. SUBDCA: Expected less as not all DCA code required subdca coded.
9. VEHICLE: Some accident can involve multiple vehicles.

### Missing Values Count

In [3]:
# Check for missing values in each dataset
def missing_value_summary(df):
    total_rows = len(df)
    summary = {}
    for col in df.columns:
        n_missing_nan = df[col].isna().sum()
        n_missing_blank = df[col].astype(str).str.strip().eq("").sum()
        n_missing = n_missing_nan + n_missing_blank
        summary[col] = {
            "missing_count": n_missing,
            "missing_percent": round((n_missing / total_rows) * 100, 2)
        }
    return pd.DataFrame(summary).T.sort_values("missing_percent", ascending=False)

In [4]:
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)

In [5]:
# Generate missing value summary for each dataset
missing_value_summaries = {}
for name, df in datasets.items():
    missing_value_summaries[name] = missing_value_summary(df)
# Display in the dataframe
for name, summary in missing_value_summaries.items():
    print(f"Missing values in {name} dataset:")
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    display(summary)

Missing values in ACCIDENT dataset:


,missing_count,missing_percent
PAGE,6894.0,3.38
DIRECTORY,6894.0,3.38
EDITION,6894.0,3.38
GRID_REFERENCE_X,6894.0,3.38
GRID_REFERENCE_Y,6894.0,3.38
ACCIDENTTIME,0.0,0.00
ACCIDENT_TYPE,0.0,0.00
Accident Type Desc,0.0,0.00
DCA_CODE,0.0,0.00
Day Week Description,0.0,0.00


Missing values in ACCIDENT_CHAINAGE dataset:


,missing_count,missing_percent
Node Id,1.0,0.0
Route No,1.0,0.0
Chainage Seq,1.0,0.0
Route Link No,1.0,0.0
Chainage,1.0,0.0


Missing values in ACCIDENT_EVENT dataset:


...

,missing_count,missing_percent
ACCIDENT_NO,0.0,0.0
SURFACE_COND,0.0,0.0
Surface Cond Desc,0.0,0.0
SURFACE_COND_SEQ,0.0,0.0


Missing values in SUBDCA dataset:


,missing_count,missing_percent
ACCIDENT_NO,0.0,0.0
SUB_DCA_CODE,5.0,0.0
SUB_DCA_SEQ,0.0,0.0
Sub Dca Code Desc,0.0,0.0


Missing values in VEHICLE dataset:


,missing_count,missing_percent
VEHICLE_POWER,365240.0,100.00
CUBIC_CAPACITY,321920.0,88.14
VEHICLE_WEIGHT,319083.0,87.36
CARRY_CAPACITY,294848.0,80.73
CONSTRUCTION_TYPE,92793.0,25.41
SEATING_CAPACITY,85300.0,23.35
NO_OF_WHEELS,71085.0,19.46
NO_OF_CYLINDERS,63248.0,17.32
TARE_WEIGHT,40730.0,11.15
VEHICLE_MODEL,40084.0,10.97
